In [37]:
import pandas as pd
import numpy as np
import pyodbc
from pathlib import Path

import matplotlib.pyplot as plt

import numpy as np
from scipy.interpolate import interp1d

import requests
import folium

In [2]:
# Configuración de los parámetros de conexión
server = 'localhost\\SQLEXPRESS'  # O el punto '.' que usaste en SSMS
database = 'EM_CUNBOY'

# Cadena de conexión usando Autenticación de Windows (Trusted_Connection)
conn_str = (
    f'DRIVER={{ODBC Driver 17 for SQL Server}};'
    f'SERVER={server};'
    f'DATABASE={database};'
    f'Trusted_Connection=yes;'
)


In [3]:
query = "select * from dbo.estaciones_full;"
conn = pyodbc.connect(conn_str)
raw_data = pd.read_sql(query, conn)

conn.close()

C:\Users\User\AppData\Local\Temp\ipykernel_25912\341712462.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  raw_data = pd.read_sql(query, conn)


In [4]:
query = "select * from dbo.coordenadas_estaciones;"
conn = pyodbc.connect(conn_str)
coor = pd.read_sql(query, conn)
coor.drop(["tx_id"], axis=1, inplace=True)

conn.close()

C:\Users\User\AppData\Local\Temp\ipykernel_25912\2772575436.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  coor = pd.read_sql(query, conn)


In [5]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80635 entries, 0 to 80634
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   tx_id              80635 non-null  int64         
 1   codigo_estacion    80635 non-null  object        
 2   fecha_observacion  80635 non-null  datetime64[ns]
 3   sensor_pre         80635 non-null  object        
 4   sensor_vel         80635 non-null  object        
 5   sensor_dir         80635 non-null  object        
 6   sensor_tem         80635 non-null  object        
 7   presion            80635 non-null  float64       
 8   velocidad          80635 non-null  float64       
 9   direccion          80635 non-null  float64       
 10  temperatura        80635 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(5)
memory usage: 6.8+ MB


# explo

## coordenadas

In [6]:
coor.apply(lambda df: f"{df["codigo_estacion"]}: ( {df["latitud"]} , {df["longitud"]} )", axis=1)

0     0035085080: ( 5.099 , -73.436 )
1     0035080050: ( 5.096 , -73.053 )
2     0024015300: ( 5.643 , -73.518 )
3     0035025060: ( 4.587 , -73.996 )
4     0024035503: ( 5.818 , -72.844 )
                   ...               
79     0021206980: ( 4.785 , -73.87 )
80    0023125160: ( 5.647 , -74.071 )
81    2401500080: ( 5.634 , -73.386 )
82    0021205660: ( 4.579 , -74.523 )
83    2403500084: ( 5.758 , -72.795 )
Length: 84, dtype: object

## registro fechas

In [7]:
df = raw_data.copy()
df.drop("tx_id", axis=1, inplace=True)

In [8]:
print(df.shape)
print("-"*50)
cols = ["sensor_pre", "sensor_vel", "sensor_dir", "sensor_tem"]
for col in cols:
    print(df[col].value_counts())
    col_0 = df[col].value_counts().index[0]
    df = df[df[col] == col_0].copy()
    print("-"*50)
print(df.shape)

(80635, 10)
--------------------------------------------------
sensor_pre
0255    70567
0258    10068
Name: count, dtype: int64
--------------------------------------------------
sensor_vel
0103    69123
0111     1444
Name: count, dtype: int64
--------------------------------------------------
sensor_dir
0104    69123
Name: count, dtype: int64
--------------------------------------------------
sensor_tem
0068    68401
0071      722
Name: count, dtype: int64
--------------------------------------------------
(68401, 10)


In [9]:
estaciones = df["codigo_estacion"].unique()

In [10]:
# df = df.sort_values(["codigo_estacion", "fecha_observacion"]).reset_index(drop=True)
df = df.sort_values(["fecha_observacion", "codigo_estacion"]).reset_index(drop=True)

In [11]:
df["fecha_observacion"].diff().unique()

<TimedeltaArray>
[NaT, '0 days 00:00:00', '0 days 00:10:00', '0 days 01:00:00',
 '0 days 00:40:00']
Length: 5, dtype: timedelta64[ns]

In [12]:
# fecha_minima = pd.to_datetime("2025-12-15")
# df = df[df["fecha_observacion"] >= fecha_minima]

In [13]:
estaciones_ok = list()
n = 1
for estacion in estaciones:
    min_date = df.loc[df["codigo_estacion"] == estacion, "fecha_observacion"].min()
    max_date = df.loc[df["codigo_estacion"] == estacion, "fecha_observacion"].max()
    diff_temp = df.loc[df["codigo_estacion"] == estacion, "fecha_observacion"].diff().unique()
    rows_temp = df.loc[df["codigo_estacion"] == estacion].shape[0]
    # if len(diff_temp) == 2: 
    # if rows_temp == 744 and not estacion in ["0024035250", "0024035360"]:
    # if rows_temp == 744:
    # if rows_temp == 408:
    # if min_date <= pd.to_datetime("2025-12-14") and len(diff_temp) == 2:
    # if rows_temp == 408 and not estacion in ["0024035250"]:
    if True:
        print(n , estacion)
        estaciones_ok.append(estacion)
        for diff in diff_temp: print(">", diff)
        print("cantidad de registros: ", rows_temp)
        print("fecha min: ", min_date)
        print("fecha max: ", max_date)
        print("-"*50)
        n += 1

1 0021205518
> NaT
> 0 days 01:00:00
> 0 days 02:00:00
> 0 days 03:00:00
> 0 days 11:00:00
> 0 days 00:10:00
> 0 days 06:00:00
> 0 days 00:30:00
> 0 days 00:50:00
cantidad de registros:  796
fecha min:  2025-12-01 00:00:00
fecha max:  2025-12-31 23:00:00
--------------------------------------------------
2 0024015300
> NaT
> 0 days 01:00:00
cantidad de registros:  744
fecha min:  2025-12-01 00:00:00
fecha max:  2025-12-31 23:00:00
--------------------------------------------------
3 0021205521
> NaT
> 0 days 01:00:00
> 0 days 03:00:00
> 0 days 10:00:00
> 0 days 02:00:00
> 0 days 00:10:00
> 0 days 06:00:00
cantidad de registros:  750
fecha min:  2025-12-01 00:00:00
fecha max:  2025-12-31 23:00:00
--------------------------------------------------
4 0021205524
> NaT
> 0 days 00:10:00
> 0 days 00:40:00
> 0 days 10:00:00
> 0 days 01:10:00
> 0 days 01:00:00
> 0 days 00:30:00
> 0 days 02:00:00
> 3 days 17:00:00
> 0 days 06:00:00
> 0 days 00:20:00
cantidad de registros:  2986
fecha min:  2025

# Elevación

In [38]:
def obtener_alturas_batch(coordenadas):
    """
    Consulta la elevación de múltiples puntos usando la API de Open-Elevation.
    coordenadas: Lista de diccionarios [{'latitude': lat, 'longitude': lon}, ...]
    """
    url = "https://api.open-elevation.com/api/v1/lookup"
    payload = {"locations": coordenadas}
    
    try:
        response = requests.post(url, json=payload, timeout=20)
        if response.status_code == 200:
            return response.json()['results']
    except Exception as e:
        print(f"Error al consultar la API: {e}")
    return None

In [39]:
puntos = list()
for idx in range(len(coor)):
    puntos.append({"latitude": coor.loc[idx, "latitud"], "longitude": coor.loc[idx, "longitud"]})
    ...

In [40]:
resultados = obtener_alturas_batch(puntos)

In [48]:
resultados_dics = {"latitud":list(), "longitud":list(), "altura":list()}
resultados

for resultado in resultados:
    resultados_dics["latitud"].append(resultado["latitude"])
    resultados_dics["longitud"].append(resultado["longitude"])
    resultados_dics["altura"].append(resultado["elevation"])

In [50]:
coor_alt = pd.DataFrame(resultados_dics)

In [51]:
print(f"rows: {df.shape[0]:,}")
wsdata = df.loc[df["codigo_estacion"].isin(estaciones_ok)].copy()
wsdata.drop(["sensor_pre", "sensor_vel", "sensor_dir", "sensor_tem"], axis=1, inplace=True)
print(f"rows: {wsdata.shape[0]:,}")
print(f"{100 * wsdata.shape[0] / df.shape[0]:,.2f} %")

rows: 68,401
rows: 68,401
100.00 %


# WSDATA

In [16]:
wsdata.head()

,codigo_estacion,fecha_observacion,presion,velocidad,direccion,temperatura
0,0021195120,2025-12-01,855.3000,1.300000,301.0000,19.400000
1,0021195170,2025-12-01,718.9000,2.500000,141.0000,12.900000
2,0021195190,2025-12-01,783.8000,1.700000,134.0000,14.000000
3,0021205513,2025-12-01,752.0579,0.954600,322.3283,11.941260
4,0021205515,2025-12-01,702.6863,1.216933,196.6517,8.722219


In [17]:
wsdata = wsdata.sort_values(["codigo_estacion", "fecha_observacion"]).reset_index(drop=True)

In [52]:
wsdata["longitud"] = wsdata["codigo_estacion"].map(dict(zip(coor["codigo_estacion"], coor["longitud"])))
wsdata["latitud"] = wsdata["codigo_estacion"].map(dict(zip(coor["codigo_estacion"], coor["latitud"])))

In [54]:
wsdata = pd.merge(
    left=wsdata,
    right=coor_alt,
    how="left",
    on=["latitud", "longitud"]
)

In [55]:
wsdata.drop_duplicates(inplace=True)
wsdata.reset_index(drop=True, inplace=True)

In [56]:
wsdata.head()

,codigo_estacion,fecha_observacion,presion,velocidad,direccion,temperatura,longitud,latitud,altura
0,0021195120,2025-12-01,855.3000,1.300000,301.0000,19.400000,-74.393,4.393,1423.0
1,0021195170,2025-12-01,718.9000,2.500000,141.0000,12.900000,-74.398,3.994,3017.0
2,0021195190,2025-12-01,783.8000,1.700000,134.0000,14.000000,-74.308,4.307,2175.0
3,0021205513,2025-12-01,752.0579,0.954600,322.3283,11.941260,-74.184,4.505,3068.0
4,0021205515,2025-12-01,702.6863,1.216933,196.6517,8.722219,-74.174,4.700,2542.0


In [57]:
wsdata["fecha_observacion"].sort_values().diff().value_counts()

fecha_observacion
0 days 00:00:00    64015
0 days 00:10:00     4369
0 days 01:00:00       15
0 days 00:40:00        1
Name: count, dtype: int64

In [59]:
fecha_inicio = wsdata["fecha_observacion"].min()
fecha_fin = wsdata["fecha_observacion"].max()
print(f"{fecha_inicio=} , {fecha_fin=}")
rango_tiempo = pd.date_range(fecha_inicio, fecha_fin, freq="10min").to_list()
estaciones_registradas = wsdata["codigo_estacion"].unique()
len(estaciones_registradas)


fecha_inicio=Timestamp('2025-12-01 00:00:00') , fecha_fin=Timestamp('2025-12-31 23:50:00')


73

# Completar fechas con NaN

In [60]:
estaciones_registradas = wsdata["codigo_estacion"].unique()
estaciones = list()
latitudes = list()
longitudes = list()
alturas = list()

for estacion in estaciones_registradas:
    estaciones += [estacion]*len(rango_tiempo)
    latitudes += [wsdata.loc[wsdata["codigo_estacion"] == estacion, "latitud"].unique()[0]]*len(rango_tiempo)
    longitudes += [wsdata.loc[wsdata["codigo_estacion"] == estacion, "longitud"].unique()[0]]*len(rango_tiempo)
    alturas += [wsdata.loc[wsdata["codigo_estacion"] == estacion, "altura"].unique()[0]]*len(rango_tiempo)
    
    ...

fechas = rango_tiempo * len(estaciones_registradas)

In [61]:
base_estacion_fechas = pd.DataFrame({"codigo_estacion":estaciones, "fecha_observacion":fechas, "latitud":latitudes, "longitud":longitudes, "altura":alturas})
wsdata_full = pd.merge(left=base_estacion_fechas, right=wsdata, on=["codigo_estacion", "fecha_observacion", "latitud", "longitud", "altura"], how="left")

In [62]:
wsdata_full.head()

,codigo_estacion,fecha_observacion,latitud,longitud,altura,presion,velocidad,direccion,temperatura
0,0021195120,2025-12-01 00:00:00,4.393,-74.393,1423.0,855.3,1.3,301.0,19.4
1,0021195120,2025-12-01 00:10:00,4.393,-74.393,1423.0,NaN,NaN,NaN,NaN
2,0021195120,2025-12-01 00:20:00,4.393,-74.393,1423.0,NaN,NaN,NaN,NaN
3,0021195120,2025-12-01 00:30:00,4.393,-74.393,1423.0,NaN,NaN,NaN,NaN
4,0021195120,2025-12-01 00:40:00,4.393,-74.393,1423.0,NaN,NaN,NaN,NaN


In [63]:
wsdata_full = wsdata_full.sort_values(["codigo_estacion", "fecha_observacion"]).reset_index(drop=True)

In [64]:
wsdata_full["vel_u"] = wsdata_full.apply(lambda df: df["velocidad"] * np.sin(np.pi*(df["direccion"]/180)), axis=1)
wsdata_full["vel_v"] = wsdata_full.apply(lambda df: df["velocidad"] * np.cos(np.pi*(df["direccion"]/180)), axis=1)

In [65]:
wsdata_full["segundos"] = (wsdata_full["fecha_observacion"] - fecha_inicio) / np.timedelta64(1, 's')

In [66]:
wsdata_full = wsdata_full[[
    'codigo_estacion',
    'latitud',
    'longitud',
    'altura',
    'segundos',
    'presion',
    'vel_u',
    'vel_v',
    'temperatura',
]].copy()

In [114]:
wsdata_full.sample(5)

,codigo_estacion,latitud,longitud,altura,segundos,presion,vel_u,vel_v,temperatura
304135,2403500083,5.524,-73.077,2753.0,349800.0,NaN,NaN,NaN,NaN
152141,0024015508,4.308,-74.282,2432.0,219000.0,NaN,NaN,NaN,NaN
195526,0024035503,5.818,-72.844,2529.0,2144400.0,NaN,NaN,NaN,NaN
128382,0023125100,5.515,-73.943,2194.0,2034000.0,785.8000,-0.895393,-0.090951,14.20000
57886,0021205526,4.691,-74.389,2062.0,2590800.0,798.2363,-0.901534,0.447525,15.26542


In [116]:
path_save_wsdata = Path().cwd().parent / "data" / "raw" / "em_cunboy_251212_251231.parquet"
path_save_wsdata_full = Path().cwd().parent / "data" / "raw" / "em_cunboy_251212_251231_full.parquet"
wsdata.to_parquet(path_save_wsdata, engine="fastparquet", index=False)
wsdata_full.to_parquet(path_save_wsdata_full, engine="fastparquet", index=False)